In [ ]:
import json
import os
import requests
import holoviews as hv
from holoviews.operation.datashader import datashade
import matplotlib.pyplot as plt
import xarray

hv.extension('bokeh', width=100)

## Openeo API Demo 2

### Microservice Architektur

<img src="./microservices.png" height=70% width=70%></img>

### Vorbereitung
Bevor ich die unten stehenden Docker Befehle ausführen konnte habe ich folgendes gemacht:

1. Github Repository klonen

Ich habe das Github Repository [FelixGI1516/GeosSoftII_Frontend](https://github.com/FelixGI1516/GeoSoftII_Frontend) mittels `git clone` geklont.

2. Docker File ausführen
Ich bin in den Ordner mit dem Dockerfile gegangen und habe ein Docker image namens myserver gebaut:  
```
docker build -t myserver ./
```

3. Docker Container erzeugen
Dann habe ich einen Container basierend auf myserver erzeugt:
```
docker run --name fe_server_i -p 80:80/ myserver
```
Wichtig: Nicht den Port im Befehl vergessen (Ist mir passiert :])

In [ ]:
! docker start fe_server_i

### Die Antworten des Servers in der Übersicht:

In [ ]:
#Load in Capabilities Response JSON
x = requests.get('http://localhost/api/v1/')
capabilities = 'Capabilities:\n \n'+ json.dumps(x.json(),indent=1) + '\n \n'

print(capabilities)

In [ ]:
#Load in Collections Response JSON
y = requests.get('http://localhost/api/v1/collections')
collections  = 'Collections:\n \n'+ json.dumps(y.json(),indent=1) + '\n \n'

print(collections)

In [ ]:
#Load in Processes Response JSON
z = requests.get('http://localhost/api/v1/processes')
processes  = 'Processes:\n \n'+ json.dumps(z.json(),indent=1) + '\n \n'

print(processes)

In [ ]:
#Load in Processes Response JSON
a = requests.get('http://localhost/api/v1/jobs')
jobs  = 'Jobs:\n \n'+ json.dumps(a.json(),indent=1) + '\n \n'
#Provozieren eines Errors
ae = requests.get('http://localhost/api/v2/jobs')
error = 'Error:\n \n'+ json.dumps(ae.json(),indent=1) + '\n \n'
print(jobs, error)

### Backend Validator:


In [ ]:
#Load in wellknown Response JSON
b = requests.get('http://localhost/api/v1/.well-known/openeo')
well_know  = 'Processes:\n \n'+ json.dumps(b.json(),indent=1) + '\n \n'

print(well_know)

In [ ]:
os.chdir("./validator")

In dem Ordner in dem wir uns nun befinden ist eine Datei namens `validae.cmd`.\
Diese führen wir aus und erhalten eine neue JSON-Datei namens `validator_output`.\
Das Ergebnis ist ein JSON, welches die Kompabilität unserer Endpunkte mit den Vorschriften von openeo vergleicht und bewertet.

Die Voraussetzungen für die Installation des findet ihr Backend Validator [hier](https://github.com/Open-EO/openeo-backend-validator)

In [ ]:
! validate
with open("validator_output.json") as v_output:
    val = json.load(v_output)
    print(json.dumps(val,indent=1))

In [ ]:
! docker stop fe_server_i

In [ ]:
os.chdir("..")

### NDVI Demo

Das Problem ist das Datenformat SAFE in xarray nicht ohne Weiteres nutzbar ist.
Deshalb haben wir eine Methode entwicklet, die mittels einiger Zwischenschritte die Umwandlung in das lesbare NetCDF Format erlaubt.

Diese Zelle haben wir in der Demo gezeigt, allerdings läuft sie sehr lange und hatte wir hatten etwas Schwierigkeiten sie wieder zum Laufen zu bringen. 
Deshalb wird hier nur noch auf den Ergebnis Datacube zugegriffen.
```python
#Umwandlung der SAFE Datei und Erzeugung des NetCDF Datacube
%run NDVI
```

In [ ]:
os.chdir('../demodata')

In [ ]:
# Öffnen des Datensatzes und Repräsentation als Grauwert Bild
DS = xarray.open_dataset('datacube_NDVI.nc')
DS
ndvi_ds = hv.Dataset(DS, kdims=['null','Band', 'null','X', 'Y'])
ndvi = ndvi_ds.to(hv.QuadMesh, kdims=["X", "Y"])
#%opts Greys [width=900 height=600]
datashade(ndvi, precompute=True, cmap=plt.cm.Greens)

## SST Demo

Die SST Berechnung wurde mit Dask verfeinert und kann mit nun eine Räumliche Auswahl treffen. 
Dies geschieht ünber einen neuen Parameter der eine Bounding Box von Koordinaten entgegen nihmt.

Außerdem wurde die visuelle Repräsentation des Ergbenis überarbeitet. Diese kann in bestimmten Maßen ausgegeben werden.

In [ ]:
os.chdir("../Demo_II/")

### Setup:

In [ ]:
%run SST

In [ ]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)
client

In [ ]:
os.chdir("../demodata")

### Die verbesserte Visualisierung:

In [ ]:
%%time

x = mean_sst(['1989-01-01','1989-01-01'])

# visualisieren

sst_ds_day = hv.Dataset(x, kdims=['lon', 'lat'])
sst_day = sst_ds_day.to(hv.QuadMesh, kdims=["lon", "lat"])
%opts RGB [width=900 height=600]
datashade(sst_day, precompute=True, cmap=plt.cm.RdBu_r)

### Räumliche Auswahl der Nord- und Ostsee:

In [ ]:
x = mean_sst(['1989-10-01','1989-11-01'], [0, 50, 30, 75])

# visualisieren

# lon werte von 0 - 360 in -180 - 180 umwandeln
lon = x['lon'].values
lon = [min360(item) for item in lon]

# lon Werte im dataset durch neue ersetzten, sonst kommt was komisches raus
x = x.assign_coords(coords={"lat": x['lat'], "lon": lon})

sst_x = hv.Dataset(x, kdims=['lon', 'lat'])
sst_x = sst_x.to(hv.QuadMesh, kdims=["lon", "lat"])
%opts RGB [width=900 height=600]
datashade(sst_x, precompute=True, cmap=plt.cm.RdBu_r)